In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge,BatchNormalization
from keras.preprocessing import sequence

import numpy as np
from vgg16 import Vgg16

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import PIL.Image

import json
from tqdm import tqdm

from keras.optimizers import SGD, RMSprop, Adam

from utils import *
import cPickle as pickle
from matplotlib import pyplot as plt

from itertools import compress

import shutil
import string

import collections
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

import re
from numpy.random import random, permutation, randn, normal 

import os
import preprocessing as preproc


In [ ]:
save_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/"

annotation_path = save_path +"raw_annotations/captions_val2014.json"

images_path = save_path+"raw_images/val2014"
image_data_arr_path = save_path+"imageDataArr/"
images_concat_t_path = save_path+"imagesConcatT/"
captions_path = save_path+"captions/"
temp_save_path = save_path+"temp/"
model_path = save_path+"models/"
images_vgg_features_path = save_path + "images_vgg_features/"

train_path = save_path + "train/"
test_path = save_path + "test/"


images_concat_folder = "images_concat/"
images_vgg_4096_folder = "images_vgg_4096/"
captions_folder = "captions/"
indexed_captions_folder = "indexed-captions/"
indexed_future_words_folder = "indexed-future-words/"
glove_folder = "glove/"

misc_folder = "misc/"

batch_folder = "batches/"

# Build Data - Data structure

In [ ]:
TEST_SIZE = 1000

In [ ]:
image_data_arr = preproc.get_image_data_arr(images_path,annotation_path)
(train_image_data_arr,test_image_data_arr) = preproc.get_train_test_data(image_data_arr,TEST_SIZE)
print("Train: %d Test %d"%(len(train_image_data_arr),len(test_image_data_arr)))
 

# Build Data - Images

In [ ]:
train_images_concat_t = preproc.construct_images_concat_t(train_image_data_arr)
test_images_concat_t = preproc.construct_images_concat_t(test_image_data_arr)

print(train_images_concat_t.shape)
print(test_images_concat_t.shape)

In [ ]:
save_array(train_path + images_concat_folder+ 'images_concat.bc', train_images_concat_t)
save_array(test_path + images_concat_folder+ 'images_concat.bc', test_images_concat_t)

# Build Data - Captions

In [ ]:
preproc.write_captions_to_disk(train_path + captions_folder, train_image_data_arr)
preproc.write_captions_to_disk(test_path + captions_folder, test_image_data_arr)